In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
import nltk
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [2]:
# Get treelib
!pip install -U treelib

# Setup Stanford Parser
!wget https://nlp.stanford.edu/software/stanford-parser-4.0.0.zip
!unzip stanford-parser-4.0.0.zip
!mkdir stanford-nlp-jars
!cp stanford-parser-4.0.0/stanford-parser.jar stanford-nlp-jars/
!cp stanford-parser-4.0.0/stanford-parser-4.0.0-models.jar stanford-nlp-jars/
!unzip stanford-nlp-jars/stanford-parser-4.0.0-models.jar -d stanford-nlp-jars/stanford-parser-4.0.0-models/
!pip
from nltk.parse import stanford
os.environ['STANFORD_PARSER'] = 'stanford-nlp-jars'
os.environ['STANFORD_MODELS'] = 'stanford-nlp-jars'
constituency_parser = stanford.StanfordParser(model_path="stanford-nlp-jars/stanford-parser-4.0.0-models/edu/stanford/nlp/models/lexparser/englishPCFG.ser.gz")
dependency_parser = stanford.StanfordDependencyParser()


# Setup Stanford POS Tagger
!wget https://nlp.stanford.edu/software/stanford-tagger-4.1.0.zip
!unzip stanford-tagger-4.1.0.zip
!cp stanford-postagger-full-2020-08-06/stanford-postagger-4.1.0.jar stanford-nlp-jars/
!cp stanford-postagger-full-2020-08-06/models/english-bidirectional-distsim.tagger stanford-nlp-jars/

from nltk.tag.stanford import StanfordPOSTagger
pos_model = 'stanford-nlp-jars/english-bidirectional-distsim.tagger'
pos_jar   = 'stanford-nlp-jars/stanford-postagger-4.1.0.jar'
pos_tagger = StanfordPOSTagger(pos_model, pos_jar)

  Created wheel for treelib: filename=treelib-1.6.1-py3-none-any.whl size=18371 sha256=2572b9662fb6e72cd7280cd76818758b8b645b6562e6d7ab94bf1d8608f78d64
  Stored in directory: /root/.cache/pip/wheels/89/be/94/2c6d949ce599d1443426d83ba4dc93cd35c0f4638260930a53
Successfully built treelib
--2020-10-31 12:25:21--  https://nlp.stanford.edu/software/stanford-parser-4.0.0.zip
Resolving nlp.stanford.edu (nlp.stanford.edu)... 171.64.67.140
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 182493647 (174M) [application/zip]
Saving to: ‘stanford-parser-4.0.0.zip’

stanford-parser-4.0 100%[===================>] 174.04M  7.52MB/s    in 19s     

2020-10-31 12:25:41 (9.13 MB/s) - ‘stanford-parser-4.0.0.zip’ saved [182493647/182493647]

Archive:  stanford-parser-4.0.0.zip
   creating: stanford-parser-4.0.0/
  inflating: stanford-parser-4.0.0/ShiftReduceDemo.java  
  inflating: stanford-parser-4.0.0/lexparser-gui.co

--2020-10-31 12:25:50--  https://nlp.stanford.edu/software/stanford-tagger-4.1.0.zip
Resolving nlp.stanford.edu (nlp.stanford.edu)... 171.64.67.140
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 78279880 (75M) [application/zip]
Saving to: ‘stanford-tagger-4.1.0.zip’

stanford-tagger-4.1 100%[===================>]  74.65M  5.53MB/s    in 9.4s    

2020-10-31 12:26:01 (7.92 MB/s) - ‘stanford-tagger-4.1.0.zip’ saved [78279880/78279880]

Archive:  stanford-tagger-4.1.0.zip
   creating: stanford-postagger-full-2020-08-06/
  inflating: stanford-postagger-full-2020-08-06/stanford-postagger-4.1.0-sources.jar  
  inflating: stanford-postagger-full-2020-08-06/TaggerDemo2.java  
  inflating: stanford-postagger-full-2020-08-06/TaggerDemo.java  
  inflating: stanford-postagger-full-2020-08-06/build.xml  
   creating: stanford-postagger-full-2020-08-06/models/
  inflating: stanford-postagger-full-2020-08-06/mo

In [31]:
#### Test Parser
sent1 = "He teaches my daughter Maths"
sent1 = "The boy who jumped into the river saved another boy"
print(sent1)
# print("Constituency parsing")
# for t in constituency_parser.raw_parse(sent1):
# 	print(t)

print("Dependency parsing")
result = dependency_parser.raw_parse(sent1)
dep = result.__next__()

l = list(dep.triples())
print(l)
dep_parse = []
for el in l:
	dep_parse.append((el[0][0],el[2][0]))
print(dep_parse)

# #### Test POS Tagger
text = "He is sincere and honest"
words = nltk.word_tokenize(text)
tagged_words = pos_tagger.tag(words)
print(tagged_words)

The boy who jumped into the river saved another boy
Dependency parsing
[(('saved', 'VBD'), 'nsubj', ('boy', 'NN')), (('boy', 'NN'), 'det', ('The', 'DT')), (('boy', 'NN'), 'acl:relcl', ('jumped', 'VBD')), (('jumped', 'VBD'), 'nsubj', ('who', 'WP')), (('jumped', 'VBD'), 'obl', ('river', 'NN')), (('river', 'NN'), 'case', ('into', 'IN')), (('river', 'NN'), 'det', ('the', 'DT')), (('saved', 'VBD'), 'obj', ('boy', 'NN')), (('boy', 'NN'), 'det', ('another', 'DT'))]
[('saved', 'boy'), ('boy', 'The'), ('boy', 'jumped'), ('jumped', 'who'), ('jumped', 'river'), ('river', 'into'), ('river', 'the'), ('saved', 'boy'), ('boy', 'another')]
[('He', 'PRP'), ('is', 'VBZ'), ('sincere', 'JJ'), ('and', 'CC'), ('honest', 'JJ')]


In [32]:
tagdict = nltk.data.load('help/tagsets/upenn_tagset.pickle')
# print(tagdict.keys())
# Maps POS Tags
postagdict = { 'TO':'PREP', 'IN':'PREP',
        
        'VB':'VB', 'VBN':'VB', 'VBP':'VB',
        'VBG':'VB', 'VBZ':'VB', 'VBD':'VB',
        'MD':'VB',
        
        'NNS':'NN', 'NN':'NN', 'NNP':'NN',
        'NNPS':'NN', 'EX':'NN',
        
        'RB':'RB', 'RBR':'RB', 'RBS':'RB',
        'WRB':'RB',
        
        'PRP':'PR', 'PRP$':'PR', 'WP':'PR',
        'WP$':'PR',
        
        "''":'OOV', "UH":'OOV', 'LS':'OOV',
        '--':'OOV', ':':'OOV', '(':'OOV',
        ')':'OOV', '.':'OOV', ',':'OOV',
        '``':'OOV', '$':'OOV', 'FW':'OOV',
        'SYM':'OOV',
        
        'JJ':'JJ', 'JJR':'JJ', 'JJS':'JJ',
        
        'DT':'DT', 'WDT':'DT', 'PDT':'DT',
        
        'RP':'CASE',
        
        'CC':'CC',
        
        'CD':'NUM',
        
        'POS':'POS'}


In [33]:
# Obtain POS Tags of each word:
# Number each word from 0 to n-1
ind = 0
def traverse_tree_worker(tree_iter):
    global ind
    ind = 0
    for t in tree_iter:
        new_tree = t
    pos_tags = []
    words = []
    ind = 0
    
    
    def traverse_tree(tree):
        global ind
        one_subtree = False
        extract_label = False
        for i in range(len(tree)):
            subtree = tree[i]
            if type(subtree) == nltk.tree.Tree:
                one_subtree = True
                subtree = traverse_tree(subtree)
            else:
                words.append(subtree)
                subtree = ind
                ind +=1
            tree[i] = subtree

        if one_subtree == False:
            pos_tags.append(tree.label())
        return tree
    final_tree = traverse_tree(new_tree)
    iter_tree = iter(final_tree)
    return iter_tree,pos_tags, words

def copy_nltk_tree(tree_iter):
    for t in tree_iter:
        new_tree = t
    
    new_iter = iter(new_tree)
    return new_iter

In [34]:
sent1 = "The quick brown fox jumped over the lazy dog"
sent1 = "Senior boys who had exams played football on the ground"
sent1 = "The boy who jumped into the river saved another boy"
print(sent1)
# print("Constituency parsing")
parsed_sent1 = constituency_parser.raw_parse(sent1)
parsed_sent1_old = parsed_sent1
parsed_sent1_new,pos_tags, words = traverse_tree_worker(parsed_sent1)
print("Old parse")
for t in parsed_sent1_old:
	print(t)
print("New parse")
for t in parsed_sent1_new:
    print(t)

The boy who jumped into the river saved another boy
Old parse
New parse
(S
  (NP
    (NP (DT 0) (NN 1))
    (SBAR
      (WHNP (WP 2))
      (S (VP (VBD 3) (PP (IN 4) (NP (DT 5) (NN 6)))))))
  (VP (VBD 7) (NP (DT 8) (NN 9))))


In [35]:
# def traverse_tree(tree):
# 	print("tree:", tree, tree.label())
# 	for subtree in tree:
# 		if type(subtree) == nltk.tree.Tree:
# 			traverse_tree(subtree)

sent1 = "The quick brown fox jumped over the lazy dog"
sent1 = "Senior boys who had exams played football on the ground"
sent1 = "The boy who jumped into the river saved another boy"
print(sent1)
# print("Constituency parsing")
parsed_sent1 = constituency_parser.raw_parse(sent1)
parsed_sent1, pos_tags, words = traverse_tree_worker(parsed_sent1)
# for t in parsed_sent1:
# 	print(t)


def phrase_to_dependency(tree):
	h,dep_tree = convert_to_dependency_without_root(tree)
	return add_root_node(dep_tree, h)

def convert_to_dependency_without_root(tree):
	children_heads = []
	dep_tree = []
	#print(tree)
	for subtree in tree:
		if type(subtree) == nltk.tree.Tree and subtree.height() > 2:
			h, dep_subtree = convert_to_dependency_without_root(subtree)
			children_heads.append(h)
			dep_tree += dep_subtree
		elif type(subtree) == nltk.tree.Tree and subtree.height() == 2:
			h = subtree.leaves()[0]#[subtree.label(), subtree.leaves()[0]]
			children_heads.append(h)
	#print(children_heads)
	h = find_head(tree)
	#print(h)
	for m in children_heads:
		if m != h:
			dep_tree.append((h, m))
	#print(dep_tree)
	return h, dep_tree

def add_root_node(dep_tree, h):
	dep_tree.append((-1, h))
	return dep_tree

def find_head_old(possible_heads):
	h = possible_heads[0]
	for p in possible_heads:
		if more_imp(p[0],h[0]):
			h = p
	#print(h)
	return h

def find_head(tree):
	if tree.label() == 'NP':
		return find_head_of_np(tree)
	elif tree.label() == 'VP':
		return find_head_of_vp(tree)
	# elif tree.label() == 'PP':
	# 	find_head_of_pp(tree)
	elif tree.label() == 'ROOT':
		subtrees = [t for t in tree if type(t)==nltk.tree.Tree]
		return find_head_of_s(subtrees[0])
	elif tree.label() == 'S':
		return find_head_of_s(tree)
	else:
		return find_head_in_general(tree)

def find_head_of_np(np):
	noun_tags = ['NN', 'NNS', 'NNP', 'NNPS', 'PRP']
	top_level_trees = [np[i] for i in range(len(np)) if type(np[i]) is nltk.tree.Tree]
	## search for a top-level noun
	top_level_nouns = [t.leaves()[0] for t in top_level_trees if t.label() in noun_tags]
	if len(top_level_nouns) > 0:
		return top_level_nouns[-1]
	else:
		top_level_nps = [t for t in top_level_trees if t.label()=='NP']
		if len(top_level_nps) > 0:
			return find_head_of_np(top_level_nps[-1])
		else:
			nouns = [p[0] for p in np.pos() if p[1] in noun_tags]
			if len(nouns) > 0:
				return nouns[-1]
			else:
				return np.leaves()[-1]

def find_head_of_vp(vp):
	verb_tags = ['VB', 'VBD', 'VBG', 'VBN', 'VBP', 'VBZ']
	top_level_trees = [vp[i] for i in range(len(vp)) if type(vp[i]) is nltk.tree.Tree]
	top_level_verbs = [t.leaves()[0] for t in top_level_trees if t.label() in verb_tags]
	if len(top_level_verbs) > 0:
		return top_level_verbs[0]
	else:
		top_level_vps = [t for t in top_level_trees if t.label()=='VP']
		if len(top_level_vps) > 0:
			return find_head_of_vp(top_level_vps[0])
		else:
			verbs = [p[0] for p in vp.pos() if p[1] in verb_tags]
			if len(verbs) > 0:
				return verbs[0]
			else:
				return vp.leaves()[0]

def find_head_of_s(s):
	top_level_trees = [s[i] for i in range(len(s)) if type(s[i]) is nltk.tree.Tree]
	top_level_vps = [t for t in top_level_trees if t.label()=='VP']
	top_level_nps = [t for t in top_level_trees if t.label()=='NP']
	if len(top_level_vps) > 0:
		return find_head_of_vp(top_level_vps[0])
	elif len(top_level_nps) > 0:
		return find_head_of_np(top_level_nps[-1])
	else:
		return s.leaves()[0]

def find_head_in_general(s):
	top_level_trees = [s[i] for i in range(len(s)) if type(s[i]) is nltk.tree.Tree]
	top_level_vps = [t for t in top_level_trees if t.label()=='VP']
	top_level_nps = [t for t in top_level_trees if t.label()=='NP']
	top_level_s = [t for t in top_level_trees if t.label()=='S']
	if len(top_level_vps) > 0:
		return find_head_of_vp(top_level_vps[0])
	elif len(top_level_nps) > 0:
		return find_head_of_np(top_level_nps[-1])
	elif len(top_level_s) > 0:
		return find_head_of_s(top_level_s[0])	
	else:
		return s.leaves()[0]

## Not required
# def find_head_of_pp(pp):
# 	prep_tags = ['TO', 'IN']
# 	top_level_trees = [pp[i] for i in range(len(pp)) if type(pp[i]) is nltk.tree.Tree]
# 	top_level_preps = [t.leaves()[0] for t in top_level_trees if t.label() in prep_tags]
# 	if len(top_level_preps) > 0:
# 		return top_level_preps[0]
# 	else:
# 		top_level_pps = [t for t in top_level_trees if t.label()=='PP']
# 		if len(top_level_pps) > 0:
# 			return find_head_of_pp(top_level_pps[0])
# 		else:
# 			preps = [p[0] for p in pp.pos() if p[1] in prep_tags]
# 			if len(preps) > 0:
# 				return preps[0]
# 			else:
# 				return pp.leaves()[0]	

def more_imp(t1, t2):
	if t1.startswith("V") and not t2.startswith("V"):
		return True
	if t1.startswith("N") and not (t2.startswith("V") or t2.startswith("N")):
		return True

	return False

for t in parsed_sent1:
	print("Dependency parser")
	dep_tree = phrase_to_dependency(t)
	print(dep_tree)

The boy who jumped into the river saved another boy
Dependency parser
[(1, 0), (6, 5), (6, 4), (3, 6), (3, 2), (1, 3), (9, 8), (7, 9), (7, 1), (-1, 7)]


In [47]:
from treelib import Tree, Node
def printDependencyTree(lst, sent):
    '''
        Input: A list with values as (parent, child, tag)[Note: Indices]
               Sentence
        Prints the tree
    '''
    words_with_index = []
    for i in range(len(lst)):
        words_with_index.append("Dummy")
        
    i = 0
    for item in lst:
        new_word = f"{item[2]}-{sent[item[1]]}-{item[1]}"
        i+=1
        words_with_index[item[1]] = new_word
    
    tree = Tree()
    tree.create_node("Root", -1)
    for (t1, t2, t3) in lst:
        tree.create_node(words_with_index[t2], t2, parent = -1)
    for (t1, t2, t3) in lst:
        tree.move_node(t2, t1)
    tree.show()
    
sent = ['The', 'boy', 'jumped']
# lst = [(-1, 2), (2, 1), (1, 0)]
lst  = [(-1, 2, 'VB'), (2, 1, 'NSUBJ'), (1, 0, 'DET')]
printDependencyTree(lst, sent)

Root
└── VB-jumped-2
    └── NSUBJ-boy-1
        └── DET-The-0



In [37]:
printDependencyTree(dep_tree, words_with_index)

IndexError: tuple index out of range

In [38]:
children = []
for i in range(len(words)):
    children.append([])

for word_pair in new_dep_tree:
    parent = word_pair[0]
    child = word_pair[1]
    if parent != -1:
        children[parent].append(child)
print(children)

[[], [0, 3], [], [6, 2], [], [], [5, 4], [9, 1], [], [8]]


In [48]:
def addRules(dependency_tree,children,pos_tags):
    
    dep_tree_with_tags = []
    
    # Get main verb in the sentence
    main_verb = 0
    parent_cc = 0
    for words in dependency_tree:
        if words[0] == -1:
            main_verb = words[1]
        if postagdict[pos_tags[words[1]]] == 'CC':
            parent_cc = words[0]
    
    # Find the last noun after main verb
    last_noun = 0
    for i in range(main_verb, len(children)):
        if postagdict[pos_tags[i]] == 'NN':
            last_noun = i
    
    # Start tagging edges
    for words in dependency_tree:
        new_grouping = []

        new_grouping.append(words[0])
        new_grouping.append(words[1])
        pos = postagdict[pos_tags[words[1]]] 
        if words[0] != -1:
            pos_parent = postagdict[pos_tags[words[0]]]
        else:
            pos_parent = "ROOT"
            new_grouping.append("ROOT")
            dep_tree_with_tags.append(new_grouping)
            continue
            
        if pos == "DT":
            new_grouping.append("DET")
        
        elif pos == "CC":
            new_grouping.append("CC")
        
        elif pos_parent == "NN" and pos == "JJ":
            new_grouping.append("AMOD")
        
        elif pos_parent == "NN" and pos == "NUM":
            new_grouping.append("NUMMOD")
        
        elif pos_parent == "NN" and pos == "NN":
            # Resolve APPOS or NMOD
            isAppos = True
            for i in range(words[0], words[1]+1):
                if postagdict[pos_tags[i]] == "PREP":
                    isAppos = False
                    break
            if isAppos is True:
                new_grouping.append("APPOS")
            else:
                new_grouping.append("NMOD")
        
        elif pos_parent == "VB" and pos == "NN":
            ## Resolve Object and Subject
            if words[0] > words[1]:
                new_grouping.append("NSUBJ")
            elif words[1] == last_noun:
                new_grouping.append("DOBJ")
            else:
                new_grouping.append("IOBJ")
        
        elif pos_parent == "VB" and (pos == "VB" or pos == "JJ"):
            new_grouping.append("CCOMP")
        
        elif pos != "VB" and pos != "NN" and pos != "JJ" and pos != "DT":
            new_grouping.append("CASE")
        
        elif words[1] == parent_cc:
            new_grouping.append("CONJ")
        
        else:
            new_grouping.append("XCOMP")
        dep_tree_with_tags.append(new_grouping)
    return dep_tree_with_tags


dep_tree_with_tags = addRules(dep_tree, children, pos_tags)
print(dep_tree_with_tags)
printDependencyTree(dep_tree_with_tags, words)
                
            

[[1, 0, 'DET'], [6, 5, 'DET'], [6, 4, 'CASE'], [3, 6, 'IOBJ'], [3, 2, 'CASE'], [1, 3, 'XCOMP'], [9, 8, 'DET'], [7, 9, 'DOBJ'], [7, 1, 'NSUBJ'], [-1, 7, 'ROOT']]
Root
└── ROOT-saved-7
    ├── DOBJ-boy-9
    │   └── DET-another-8
    └── NSUBJ-boy-1
        ├── DET-The-0
        └── XCOMP-jumped-3
            ├── CASE-who-2
            └── IOBJ-river-6
                ├── CASE-into-4
                └── DET-the-5



In [ ]:
[[1, 0, 'DET'],
 [6, 5, 'DET'],
 [6, 4, 'OTH'],
 [3, 6, 'DOBJ'],
 [3, 2, 'OTH'],
 [1, 3, 'OTH'],
 [9, 8, 'DET'],
 [7, 9, 'DOBJ'],
 [7, 1, 'NSUBJ']]

In [28]:
print(dep_tree_with_tags)

[[1, 0, 'DET'], [6, 5, 'DET'], [6, 4, 'CASE'], [3, 6, 'IOBJ'], [3, 2, 'CASE'], [1, 3, 'XCOMP'], [9, 8, 'DET'], [7, 9, 'DOBJ'], [7, 1, 'NSUBJ'], [-1, 7, 'ROOT']]


In [41]:
print(words[7])

saved
